## 🔢 ОТВЕТ: Количество символов на барабанах

```python
reels = {
    'R1': {
        'Банан': 156, 
        'Апельсин': 79, 
        'Манго': 21, 
        'Черешня': 21, 
        'Персик': 10, 
        'Вишня': 10
    },
    'R2': {
        'Банан': 168, 
        'Апельсин': 50, 
        'Манго': 21, 
        'Черешня': 11, 
        'Персик': 10, 
        'Вишня': 10
    },
    'R3': {
        'Банан': 167, 
        'Апельсин': 58, 
        'Манго': 20, 
        'Черешня': 20, 
        'Персик': 15, 
        'Вишня': 14
    }
}

In [87]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

# Исходные данные
symbols = ['Банан', 'Апельсин', 'Манго', 'Черешня', 'Персик', 'Вишня']
payments = [5, 2, 1, 1, 0.5, 0.5]
target_rtp = 0.95

# Начальные количества символов ( начальное приближение, но тут уже вставил готовый вариант на rtp ~0.95 )
initial_counts = {
    'R1': [156, 79, 21, 21, 10, 10],
    'R2': [168, 50, 21, 11, 10, 10],
    'R3': [167, 58, 20, 20, 15, 14]
}

def calculate_metrics(counts_array):
    counts = counts_array.astype(int)
    total_counts = {
        'R1': sum(counts[:6]),
        'R2': sum(counts[6:12]),
        'R3': sum(counts[12:])
    }
    
    probs = {
        'R1': [c/total_counts['R1'] for c in counts[:6]],
        'R2': [c/total_counts['R2'] for c in counts[6:12]],
        'R3': [c/total_counts['R3'] for c in counts[12:]]
    }
    
    rtp = 0
    var = 0
    for i in range(6):
        p = probs['R1'][i] * probs['R2'][i] * probs['R3'][i]
        rtp += p * payments[i]
    
    for i in range(6):
        p = probs['R1'][i] * probs['R2'][i] * probs['R3'][i]
        var += p * (payments[i] - rtp)**2
    
    return rtp, var, counts, probs

def objective(x):
    x_ints = np.round(x).astype(int)
    rtp, var, _, _ = calculate_metrics(x_ints)
    
    rtp_penalty = (rtp - target_rtp)**2 * 1000
    var_penalty = var * 0.1
    negative_penalty = sum(np.where(x < 1, (1 - x)**2, 0)) * 100
    
    return rtp_penalty + var_penalty + negative_penalty

x0 = np.array(initial_counts['R1'] + initial_counts['R2'] + initial_counts['R3'])
bounds = [(1, 500) for _ in range(18)]

result = minimize(objective, x0, bounds=bounds, method='L-BFGS-B', options={'maxiter': 1000, 'eps': 1})

optimized_counts = np.round(result.x).astype(int)
rtp, var, counts, probs = calculate_metrics(optimized_counts)

optimized_reels = {
    'R1': dict(zip(symbols, counts[:6])),
    'R2': dict(zip(symbols, counts[6:12])),
    'R3': dict(zip(symbols, counts[12:]))
}

# Создание таблицы с правильным количеством столбцов
table_data = []
for i, symbol in enumerate(symbols):
    p = probs['R1'][i] * probs['R2'][i] * probs['R3'][i]
    table_data.append({
        'Символ': symbol,
        'R1 Количество': optimized_reels['R1'][symbol],
        'R1 Вероятность': f"{probs['R1'][i]:.4f}",
        'R2 Количество': optimized_reels['R2'][symbol],
        'R2 Вероятность': f"{probs['R2'][i]:.4f}",
        'R3 Количество': optimized_reels['R3'][symbol],
        'R3 Вероятность': f"{probs['R3'][i]:.4f}",
        'Вер. комбинации': f"{p:.6f}",
        'Выплата': payments[i],
        'Вклад RTP': f"{p * payments[i]:.6f}"
    })

df = pd.DataFrame(table_data)

# Добавляем итоговую строку с правильным количеством значений
df.loc[len(df)] = [
    'ИТОГО',
    '-', '-', '-', '-', '-', '-',
    '-',
    '-',
    f"{sum(float(row['Вклад RTP']) for row in table_data):.6f}"
]

print("Оптимизированные параметры:")
print(f"\nОбщий RTP: {rtp:.4f} (цель: {target_rtp})")
print(f"Дисперсия: {var:.6f}")
print(f"Стандартное отклонение: {np.sqrt(var):.6f}")

print("\nОптимизированные количества символов:")
for reel in optimized_reels:
    print(f"\n{reel}:")
    for symbol, count in optimized_reels[reel].items():
        print(f"  {symbol}: {count} шт.")

df

Оптимизированные параметры:

Общий RTP: 0.9483 (цель: 0.95)
Дисперсия: 3.058385
Стандартное отклонение: 1.748824

Оптимизированные количества символов:

R1:
  Банан: 156 шт.
  Апельсин: 79 шт.
  Манго: 21 шт.
  Черешня: 21 шт.
  Персик: 10 шт.
  Вишня: 10 шт.

R2:
  Банан: 168 шт.
  Апельсин: 50 шт.
  Манго: 21 шт.
  Черешня: 11 шт.
  Персик: 10 шт.
  Вишня: 10 шт.

R3:
  Банан: 167 шт.
  Апельсин: 58 шт.
  Манго: 20 шт.
  Черешня: 20 шт.
  Персик: 15 шт.
  Вишня: 14 шт.


,Символ,R1 Количество,R1 Вероятность,R2 Количество,R2 Вероятность,R3 Количество,R3 Вероятность,Вер. комбинации,Выплата,Вклад RTP
0,Банан,156,0.5253,168,0.6222,167,0.5680,0.185645,5.0,0.928224
1,Апельсин,79,0.2660,50,0.1852,58,0.1973,0.009718,2.0,0.019435
2,Манго,21,0.0707,21,0.0778,20,0.0680,0.000374,1.0,0.000374
3,Черешня,21,0.0707,11,0.0407,20,0.0680,0.000196,1.0,0.000196
4,Персик,10,0.0337,10,0.0370,15,0.0510,0.000064,0.5,0.000032
5,Вишня,10,0.0337,10,0.0370,14,0.0476,0.000059,0.5,0.000030
6,ИТОГО,-,-,-,-,-,-,-,-,0.948291


In [88]:
import numpy as np
from collections import defaultdict
import pandas as pd

# Оптимизированные параметры барабанов
reels = {
    'R1': {'Банан': 156, 'Апельсин': 79, 'Манго': 21, 'Черешня': 21, 'Персик': 10, 'Вишня': 10},
    'R2': {'Банан': 168, 'Апельсин': 50, 'Манго': 21, 'Черешня': 11, 'Персик': 10, 'Вишня': 10},
    'R3': {'Банан': 167, 'Апельсин': 58, 'Манго': 20, 'Черешня': 20, 'Персик': 15, 'Вишня': 14}
}

# Выплаты за комбинации
payments = {
    'Банан': 5,
    'Апельсин': 2,
    'Манго': 1,
    'Черешня': 1,
    'Персик': 0.5,
    'Вишня': 0.5
}

# Подготовка данных для симуляции
def prepare_reel(reel, shuffle=True):
    symbols = []
    for symbol, count in reel.items():
        symbols.extend([symbol] * count)
    if shuffle:
        np.random.shuffle(symbols)
    return symbols

# Создаем барабаны
reel1 = prepare_reel(reels['R1'])
reel2 = prepare_reel(reels['R2'])
reel3 = prepare_reel(reels['R3'])

# Симуляция
np.random.seed(42)
num_spins = 1_000_000
total_bet = num_spins
total_win = 0

# Статистика
combinations = defaultdict(int)
symbol_stats = defaultdict(lambda: defaultdict(int))

for _ in range(num_spins):
    # Выбираем случайные символы
    s1 = np.random.choice(reel1)
    s2 = np.random.choice(reel2)
    s3 = np.random.choice(reel3)
    
    # Записываем статистику по символам
    symbol_stats['R1'][s1] += 1
    symbol_stats['R2'][s2] += 1
    symbol_stats['R3'][s3] += 1
    
    # Проверяем комбинацию
    if s1 == s2 == s3:
        win = payments[s1]
        total_win += win
        combinations[s1] += 1

# Расчёт RTP
rtp = total_win / total_bet

# Вывод результатов
print(f"Результаты после {num_spins:,} спинов:")
print(f"Общий выигрыш: {total_win:,.2f} (ставок: {total_bet:,})")
print(f"RTP: {rtp:.4f} ({rtp*100:.2f}%)")

# Статистика по комбинациям
print("\nСтатистика выпадения комбинаций:")
comb_stats = []
for symbol in payments:
    count = combinations.get(symbol, 0)
    prob = count / num_spins
    expected_payment = prob * payments[symbol]
    comb_stats.append({
        'Символ': symbol,
        'Количество': count,
        'Частота': f"{prob:.6f}",
        'Выплата': payments[symbol],
        'Вклад в RTP': f"{expected_payment:.6f}"
    })

df_comb = pd.DataFrame(comb_stats)
df_comb.loc[len(df_comb)] = [
    'ИТОГО',
    sum(combinations.values()),
    f"{sum(combinations.values())/num_spins:.6f}",
    '-',
    f"{rtp:.6f}"
]
print(df_comb.to_string(index=False))

# Статистика по символам на барабанах
print("\nЧастота выпадения символов на барабанах:")
reel_stats = []
for reel in ['R1', 'R2', 'R3']:
    total = sum(symbol_stats[reel].values())
    for symbol in payments:
        count = symbol_stats[reel][symbol]
        reel_stats.append({
            'Барабан': reel,
            'Символ': symbol,
            'Количество': count,
            'Частота': f"{count/total:.6f}",
            'Ожидаемая': f"{reels[reel][symbol]/sum(reels[reel].values()):.6f}"
        })

df_reels = pd.DataFrame(reel_stats)
df_reels

Результаты после 1,000,000 спинов:
Общий выигрыш: 946,021.50 (ставок: 1,000,000)
RTP: 0.9460 (94.60%)

Статистика выпадения комбинаций:
  Символ  Количество  Частота Выплата Вклад в RTP
   Банан      185156 0.185156     5.0    0.925780
Апельсин        9781 0.009781     2.0    0.019562
   Манго         407 0.000407     1.0    0.000407
 Черешня         216 0.000216     1.0    0.000216
  Персик          56 0.000056     0.5    0.000028
   Вишня          57 0.000057     0.5    0.000029
   ИТОГО      195673 0.195673       -    0.946021

Частота выпадения символов на барабанах:


,Барабан,Символ,Количество,Частота,Ожидаемая
0,R1,Банан,525478,0.525478,0.525253
1,R1,Апельсин,265572,0.265572,0.265993
2,R1,Манго,70830,0.070830,0.070707
3,R1,Черешня,70732,0.070732,0.070707
4,R1,Персик,33510,0.033510,0.033670
5,R1,Вишня,33878,0.033878,0.033670
6,R2,Банан,621802,0.621802,0.622222
7,R2,Апельсин,185183,0.185183,0.185185
8,R2,Манго,78162,0.078162,0.077778
9,R2,Черешня,40453,0.040453,0.040741
